In [ ]:
# this from the Colab Notebook is not running here
    # %%bash
    # uv pip install -q llama-index-core
    # uv pip install -q llama-index-llms-groq
    # uv pip install -q llama-index-readers-file
    # uv pip install -q llama-index-embeddings-huggingface
    # uv pip install -q llama-index-embeddings-instructor



! pip install -q llama-index-core
! pip install -q llama-index-llms-groq
! pip install -q llama-index-readers-file
! pip install -q llama-index-embeddings-huggingface
! pip install -q llama-index-embeddings-instructor

In [26]:
# Import os to have interaction with the operating system and be able to assign vairables in the os.
import os



# Find the data (just for the class example, where a txt file was copied from the Internet)

In [ ]:
# Code from Colab which needs to be adapted to be used in a local machine.
# %mkdir -p /content/data
# !wget -O /content/data/alice_in_wonderland.txt https://www.gutenberg.org/cache/epub/11/pg11.txt

from pathlib import Path
import urllib.request

# Define the download URL
url = "https://www.gutenberg.org/cache/epub/11/pg11.txt"

# Define the destination path in the current directory
destination = Path().resolve() / "alice_in_wonderland.txt"

# Download the file
urllib.request.urlretrieve(url, destination)

print(f"File downloaded to: {destination}")


File downloaded to: C:\Users\luisf\OneDrive\FERNANDO\Training\Data Science BootCamp\10_Generative_AI\Exercises\alice_in_wonderland.txt


In [27]:
destination

NameError: name 'destination' is not defined

## Load the data

### Split Documents

In [28]:
# Split the document
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

text_splitter = SentenceSplitter(chunk_size=800, chunk_overlap=150)

### Embedding

In [29]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embedding_folder = r"./embedding_model/" # "C:/Users/luisf/OneDrive/FERNANDO/Training/Data Science BootCamp/10_Generative_AI/Exercises/embedding_model/"

embeddings = HuggingFaceEmbedding(
    model_name = embedding_model, cache_folder= embedding_folder
)



2025-09-18 10:23:56,284 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


2025-09-18 10:23:59,273 - INFO - 2 prompts are loaded, with the keys: ['query', 'text']


### Creating a Vector Database

In [30]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

documents = SimpleDirectoryReader("./", required_exts=[".pdf"]).load_data() # loads only -txt documents from the active directory
vector_index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter], embed_model = embeddings
)

+ Once the database is made, you can save it to use over and over again in the future.

In [31]:
vector_index.storage_context.persist(persist_dir="./vector_index")

# Streamlit

## Build the engine

In [ ]:
#%%writefile rag_app.py


# from llama_index.llms.groq import Groq
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import StorageContext, load_index_from_storage
# from llama_index.core.chat_engine import ContextChatEngine
# from llama_index.core.memory import ChatMemoryBuffer
# from llama_index.core.base.llms.types import ChatMessage, MessageRole
# import streamlit as st

# # Secrets



# # prompt
# prefix_messages = [
#      ChatMessage(
#         role=MessageRole.SYSTEM,
#         content="You are a nice chatbot having a conversation with a human.",
#     ),
#     ChatMessage(
#         role=MessageRole.SYSTEM, content="Keep your answers short and succinct."
#     ),
#     ChatMessage(
#         role=MessageRole.SYSTEM,
#         content="You are a cooking chef specialized in south american and geman cousine.",
#     ),
#    ChatMessage(
#         role=MessageRole.SYSTEM,
#         content="You will ask the user if it wants to have the answer in a different language than english.",
#     ),
# ]



# # bot with memory
# @st.cache_resource # this is called a decorator. In this case allows to use the cache to store memory, chat memory in the case.
# def init_bot():
#     # llm
#     model = "llama-3.3-70b-versatile"

#     llm = Groq(
#         model=model,
#         token=st.secrets['GROQ_API_KEY'], # when you're running local
#     )

#     # embeddings
#     embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
#     embeddings_folder = "./embedding_model/"

#     embeddings = HuggingFaceEmbedding(
#         model_name=embedding_model,
#         cache_folder=embeddings_folder,
#     )

#     # load Vector Database
#     # allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
#     storage_context = StorageContext.from_defaults(persist_dir="./vector_index")
#     vector_index = load_index_from_storage(storage_context, embed_model=embeddings)

#     # retriever
#     retriever = vector_index.as_retriever(similarity_top_k=2)
#     # memory
#     memory = ChatMemoryBuffer.from_defaults()

#     return ContextChatEngine(
#         llm=llm, retriever=retriever, memory=memory, prefix_messages=prefix_messages
#     )


# rag_bot = init_bot()

# ##### streamlit #####

# st.title("Your Virtual Chef!")


# # Display chat messages from history on app rerun
# for message in rag_bot.chat_history:
#     with st.chat_message(message.role):
#         st.markdown(message.blocks[0].text)

# # React to user input
# if prompt := st.chat_input("For Lovers of Gastronomie!"): #:= creates a variable and store information in it at the same time.
#     # Display user message in chat message container
#     st.chat_message("human").markdown(prompt)

#     # Begin spinner before answering question so it's there for the duration
#     with st.spinner("Going down the rabbithole for answers..."):
#         # send question to chain to get answer
#         answer = rag_bot.chat(prompt)

#         # extract answer from dictionary returned by chain
#         response = answer.response

#         # Display chatbot response in chat message container
#         with st.chat_message("assistant"):
#             st.markdown(response)

Overwriting rag_app.py


## Improved Streamlit Chatbox

+ Optimization for being able to implement widgets in the chatbox.

+ Prepare the code above to be able to run the Widgets in Streamlit

In [ ]:
# %%writefile rag_app.py  # WARNING: this code is working, just trying something new below.


# from llama_index.llms.groq import Groq
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import StorageContext, load_index_from_storage
# from llama_index.core.chat_engine import ContextChatEngine
# from llama_index.core.memory import ChatMemoryBuffer
# from llama_index.core.base.llms.types import ChatMessage, MessageRole
# import streamlit as st

# # Changing the chat background for an image:
# from pathlib import Path
# import base64

# # Function to change the entire app background
# def set_app_background(image_file):
#     with open(image_file, "rb") as f:
#         encoded = base64.b64encode(f.read()).decode()

#     css = f"""
#     <style>
#     .stApp {{
#         background-image: url("data:image/jpg;base64,{encoded}");
#         background-size: cover;
#         background-position: center;
#         background-repeat: no-repeat;
#         background-attachment: fixed;
#     }}
#     </style>
#     """
#     st.markdown(css, unsafe_allow_html=True)


# def set_chat_background(image_file):
#     with open(image_file, "rb") as f:
#         encoded = base64.b64encode(f.read()).decode()
# # CSS stands for Cascading Style Sheets, and it’s the language used to style and visually design web pages.

#     css = f"""   
#     <style>
#     .stChatMessage {{
#         background-image: url("data:image/jpg;base64,{encoded}");
#         background-size: cover;
#         background-position: center;
#         background-repeat: no-repeat;
#         border-radius: 10px;
#         padding: 10px;
#     }}
#     </style>
#     """
#     st.markdown(css, unsafe_allow_html=True)

# # Full app background
# full_bg_path = Path(__file__).parent / "kitchen_wall.jpg"
# set_app_background(str(full_bg_path))

# # Call this function once at the top of your app. Assumes the file is in the same folder as the Jupyter notebook.
# image_path = Path(__file__).parent / "kitchen_background.jpg"
# set_chat_background(str(image_path))



# # Setting up system prompt options:
# prompt_options = {
#     'basic_context': (
#         'You are a chatbot with two modes: Beginner and Expert. '
#         'You are a helpful chatbot having a conversation with a human. '
#         ),
#     'Beginner': (
#         'YOU ARE NOW IN BEGINNER MODE, change your behavior if needed. '
#         'You are a helpful chatbot having a conversation with a human. '
#         'Look for easy to cook recipes. Ask always the number of people the user will cook for. Adjust your answer to that number of people. Present the ingredients first in bullet points, after that, the cooking instructions as detailed as possible and include preparation times'
#         ),
#     'Expert': (
#         'YOU ARE NOW IN EXPERT MODE, change your behavior if needed. '
#         'Search for complex recipes. Ask always the number of people the user will cook for. Adjust your answer to that number of people. Present the ingredients first in bullet points, after that, the cooking instructions with a low level of detail unless the user specify otherwise and include preparation times '
#         )
# }
# # Setting up session state to store current system prompt setting
# if 'system_prompts' not in st.session_state:
#     st.session_state['system_prompts'] = ['basic_context'] #making it a list allow it to have multiple at once

# ### INITIALIZING AND CACHING CHATBOT COMPONENTS ###

# # Function for initializing the LLM
# @st.cache_resource #the result will be cached so it only has to rerun when temp changes
# def init_llm(temp=0.01):
#     # LLM
#     return Groq(
#     model="llama-3.3-70b-versatile",
#     max_new_tokens=768,
#     temperature=temp,
#     top_p=0.95,
#     repetition_penalty=1.03,
#     token=st.secrets["GROQ_API_KEY"]
#     )

# # Function for initializing the retriever
# @st.cache_resource #the result will be cached so it only has to rerun when num_chunks changes
# def init_rag(num_chunks=2):
#     # RAG
#     embeddings = HuggingFaceEmbedding(
#         model_name="sentence-transformers/all-MiniLM-L6-v2",
#         cache_folder="./embedding_model2/",
#     )
#     storage_context = StorageContext.from_defaults(persist_dir="./vector_index")
#     vector_index = load_index_from_storage(storage_context, embed_model=embeddings)
#     return vector_index.as_retriever(similarity_top_k=num_chunks)


# # Function for initializing the chatbot memory
# @st.cache_resource #the result will be cached so it only has to run once
# def init_memory():
#     return ChatMemoryBuffer.from_defaults()


# # Function for initializing the bot with the specific settings
# @st.cache_resource #the result is cached so, unless the parameters are altered, it doesn't need to recreate the bot
# def init_bot(prefix_messages, temp=0.01, num_chunks=2):

#     # This stuff is cached and only reruns if the parameters change
#     llm = init_llm(temp) 
#     retriever = init_rag(num_chunks)
#     memory = init_memory()

#     # Takes the user selections in the session state and turns them into proper ChatMessages
#     prefix_messages = [
#         ChatMessage(
#             role=MessageRole.SYSTEM,
#             content=prompt_options[system_prompt_selection]
#             if system_prompt_selection in prompt_options
#             else system_prompt_selection
#         )
#         for system_prompt_selection in prefix_messages
#     ]

#     # Return initialized bot
#     return ContextChatEngine(
#         llm=llm, retriever=retriever, memory=memory, prefix_messages=prefix_messages
#     )

#     ##### STREAMLIT #####

# st.title("Your Virtual Chef!")


# ### TEMPERATURE SLIDER ###
# # temp = st.slider('Adjust the bot\'s creativity level', 0.0, 2.0) # decide not to activate this.
# temp = 0.1

# ### PROMPT CUSTOMIZATION ###

# # User can change the system prompts (see the dictionary above ^)
# # if new_prompt := st.selectbox('Choose an attitude for the bot:', ['Helpful', 'Unhelpful']):
# #     st.session_state['system_prompts'] = ['basic_context', new_prompt] #overwriting prompts instead of appending or swapping (for now)
# # if new_prompt := st.radio('How confident are you with cooking?:', ['Beginner', 'Expert']):
# #     st.session_state['system_prompts'] = ['basic_context', new_prompt] #overwriting prompts instead of appending or swapping (for now)


# # Create two side by side columns
# col1, col2 = st.columns(2)

# # Cooking confidence (Beginner/Expert) -- Using radio

# with col1:
#     cooking_mode = st.radio('How confident are you with cooking?:', ['Beginner', 'Expert'], key= "cooking_mode")
    


# # Recipe type (Starter / Main Dish / Dessert) — using segmented control

# with col2:
#     recipe_type = st.segmented_control(
#         "Recipe type:",
#         ["Starter", "Main Dish", "Dessert"],
#         key="recipe_type",
#         default="Main Dish"  # 👈 ensures a value immediately
#     )

# st.session_state['system_prompts'] = [
#     "basic_context",
#     st.session_state.get("cooking_mode", "Beginner"),
#     f"You are preparing a {recipe_type.lower()} recipe. Adjust your suggestions accordingly."
# ]


# ### CHAT ###

# # Initializing chatbot
# # If the parameters change, this reruns, otherwise it uses what is in the cache already
# rag_bot = init_bot(
#     prefix_messages=st.session_state['system_prompts'],
#     temp=temp,
#     num_chunks=2
# )

# # Display chat messages from history on app rerun
# for message in rag_bot.chat_history:
#     with st.chat_message(message.role):
#         st.markdown(message.blocks[0].text)


# # React to user input
# if prompt := st.chat_input('Reset the chat by typing "Goodbye"'):

#     # If user types "goodbye", reset the memory and run the app from the top again
#     if prompt.lower() == 'goodbye':
#         rag_bot.reset() # reset the bot memory
#         st.rerun() # reruns the app so that the bot is reinitialized and the chat is cleared
    
#     # Display user message in chat message container
#     st.chat_message("human").markdown(prompt)

#     # Begin spinner before answering question so it's there for the duration
#     with st.spinner("Be patient, a good meal requires always time!..."):
#         # send question to bot to get answer
#         answer = rag_bot.chat(prompt)

#         # extract answer from bot's response
#         response = answer.response

#         # Display chatbot response in chat message container
#         with st.chat_message("assistant"):
#             st.markdown(response)
# # Use streamlit run rag_app.py in Terminal to run this Python code


Overwriting rag_app.py


In [64]:
%%writefile rag_app.py


from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.chat_engine import ContextChatEngine
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.base.llms.types import ChatMessage, MessageRole
import streamlit as st

# Changing the chat background for an image:
from pathlib import Path
import base64

# Function to change the entire app background
def set_app_background(image_file):
    with open(image_file, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()

    css = f"""
    <style>
    .stApp {{
        background-image: url("data:image/jpg;base64,{encoded}");
        background-size: cover;
        background-position: center;
        background-repeat: no-repeat;
        background-attachment: fixed;
    }}
    </style>
    """
    st.markdown(css, unsafe_allow_html=True)


def set_chat_background(image_file):
    with open(image_file, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()
# CSS stands for Cascading Style Sheets, and it’s the language used to style and visually design web pages.

    css = f"""   
    <style>
    .stChatMessage {{
        background-image: url("data:image/jpg;base64,{encoded}");
        background-size: cover;
        background-position: center;
        background-repeat: no-repeat;
        border-radius: 10px;
        padding: 10px;
    }}
    </style>
    """
    st.markdown(css, unsafe_allow_html=True)

# Full app background
full_bg_path = Path(__file__).parent / "kitchen_wall.jpg"
set_app_background(str(full_bg_path))

# Call this function once at the top of your app. Assumes the file is in the same folder as the Jupyter notebook.
image_path = Path(__file__).parent / "kitchen_background.jpg"
set_chat_background(str(image_path))



# Setting up system prompt options:
prompt_options = {
    'basic_context': (
        'You are a chatbot with two modes: Beginner and Expert. '
        'You are a helpful chatbot having a conversation with a human. '
        ),
    'Beginner': (
        'YOU ARE NOW IN BEGINNER MODE, change your behavior if needed. '
        'You are a helpful chatbot having a conversation with a human. '
        'Look for easy to cook recipes. Ask always the number of people the user will cook for. Adjust your answer to that number of people. Present the ingredients first in bullet points, after that, the cooking instructions as detailed as possible and include preparation times'
        ),
    'Expert': (
        'YOU ARE NOW IN EXPERT MODE, change your behavior if needed. '
        'Search for complex recipes. Ask always the number of people the user will cook for. Adjust your answer to that number of people. Present the ingredients first in bullet points, after that, the cooking instructions with a low level of detail unless the user specify otherwise and include preparation times '
        )
}
# Setting up session state to store current system prompt setting
if 'system_prompts' not in st.session_state:
    st.session_state['system_prompts'] = ['basic_context'] #making it a list allow it to have multiple at once

### INITIALIZING AND CACHING CHATBOT COMPONENTS ###

# Function for initializing the LLM
@st.cache_resource #the result will be cached so it only has to rerun when temp changes
def init_llm(temp=0.01):
    # LLM
    return Groq(
    model="llama-3.3-70b-versatile",
    max_new_tokens=768,
    temperature=temp,
    top_p=0.95,
    repetition_penalty=1.03,
    token=st.secrets["GROQ_API_KEY"]
    )

# Function for initializing the retriever
@st.cache_resource #the result will be cached so it only has to rerun when num_chunks changes
def init_rag(num_chunks=2):
    # RAG
    embeddings = HuggingFaceEmbedding(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        cache_folder="./embedding_model2/",
    )
    storage_context = StorageContext.from_defaults(persist_dir="./vector_index")
    vector_index = load_index_from_storage(storage_context, embed_model=embeddings)
    return vector_index.as_retriever(similarity_top_k=num_chunks)


# Function for initializing the chatbot memory
@st.cache_resource #the result will be cached so it only has to run once
def init_memory():
    return ChatMemoryBuffer.from_defaults()


# Function for initializing the bot with the specific settings
@st.cache_resource
def init_bot(prefix_messages, temp=0.01, num_chunks=2):
    # Initialize components
    llm = init_llm(temp)
    retriever = init_rag(num_chunks)
    memory = init_memory()

    # Build ChatMessage list safely
    safe_prefix_messages = []
    for system_prompt_selection in prefix_messages:
        if system_prompt_selection in prompt_options:
            # Use the predefined prompt text
            content = prompt_options[system_prompt_selection]
        else:
            # Use the string as-is (dynamic prompt)
            content = system_prompt_selection

        safe_prefix_messages.append(
            ChatMessage(
                role=MessageRole.SYSTEM,
                content=content
            )
        )

    # Return initialized bot
    return ContextChatEngine(
        llm=llm,
        retriever=retriever,
        memory=memory,
        prefix_messages=safe_prefix_messages
    )


    ##### STREAMLIT #####

st.title("Your Virtual Chef!")


### PROMPT CUSTOMIZATION ###


# Create two side by side columns
col1, col2 = st.columns(2)

# Cooking confidence (Beginner/Expert) -- Using radio

with col1:
    st.radio('How confident are you with cooking? ', ['Beginner', 'Expert'], key= "cooking_mode")
    

# Recipe type (Starter / Main Dish / Dessert) — using segmented control

with col2:
    st.segmented_control(
        "Recipe type:",
        ["Starter", "Main Dish", "Dessert"],
        key="recipe_type",
        default="Main Dish"  # 👈 ensures a value immediately
    )

selected_mode = st.session_state.get("cooking_mode", "Beginner")
selected_type = st.session_state.get("recipe_type", "Main Dish")

st.session_state['system_prompts'] = [
    "basic_context",
    selected_mode,
    f"You are preparing a {selected_type.lower()} recipe. Adjust your suggestions accordingly."
]



### CHAT ###

# Initializing chatbot
# If the parameters change, this reruns, otherwise it uses what is in the cache already
rag_bot = init_bot(
    prefix_messages=st.session_state['system_prompts'],
    temp=0.5,
    num_chunks=2
)

# Display chat messages from history on app rerun
for message in rag_bot.chat_history:
    with st.chat_message(message.role):
        st.markdown(message.blocks[0].text)


# React to user input
if prompt := st.chat_input('Reset the chat by typing "Goodbye"'):

    # If user types "goodbye", reset the memory and run the app from the top again
    if prompt.lower() == 'goodbye':
        rag_bot.reset() # reset the bot memory
        st.rerun() # reruns the app so that the bot is reinitialized and the chat is cleared
    
    # Display user message in chat message container
    st.chat_message("human").markdown(prompt)

    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Be patient, a good meal requires always time!..."):
        # send question to bot to get answer
        answer = rag_bot.chat(prompt)

        # extract answer from bot's response
        response = answer.response

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)
# Use streamlit run rag_app.py in Terminal to run this Python code


Overwriting rag_app.py
